In [1]:
# !pip install scikit-learn numpy pandas pyarrow

In [2]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.preprocessing import OneHotEncoder

In [3]:
# !curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o yellow_tripdata_2023-01.parquet
# !curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -o yellow_tripdata_2023-02.parquet

In [4]:
data_j= pd.read_parquet('yellow_tripdata_2023-01.parquet')
data_f = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [5]:
len(data_j.columns)

19

In [6]:
data_j['duration_min'] = (data_j['tpep_dropoff_datetime'] - data_j['tpep_pickup_datetime']) / pd.Timedelta(minutes=1)

In [7]:
data_j['duration_min'].std()

42.59435124195458

In [8]:
outlier_mask = ((data_j['duration_min'] >= 1) & (data_j['duration_min'] <= 60))
outlier_mask.sum()/len(data_j)

0.9812202822125979

In [9]:
data_j_outlier_masked = data_j[outlier_mask]


In [21]:
# onehot_encode = pd.get_dummies(data_j_outlier_masked[['PULocationID', 'DOLocationID']], columns=['PULocationID', 'DOLocationID'])
# features = onehot_encode.values
encoder = OneHotEncoder(handle_unknown='ignore')
features = encoder.fit_transform(data_j_outlier_masked[['PULocationID', 'DOLocationID']])
targets = data_j_outlier_masked['duration_min'].values

In [22]:
features.shape

(3009173, 515)

In [23]:
model = LinearRegression().fit(features, targets)

In [24]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [25]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [26]:
np.sqrt(mean_squared_error(targets, model.predict(features)))

7.649261932396562

In [27]:
data_f['duration_min'] = (data_f['tpep_dropoff_datetime'] - data_f['tpep_pickup_datetime']) / pd.Timedelta(minutes=1)

In [28]:
outlier_mask_test = ((data_f['duration_min'] >= 1) & (data_f['duration_min'] <= 60))


In [29]:
data_f_masked = data_f[outlier_mask_test]

In [30]:
# onehot_encode_test = pd.get_dummies(data_f_masked[['PULocationID', 'DOLocationID']], columns=['PULocationID', 'DOLocationID'])

# onehot_encode_test = onehot_encode_test.reindex(columns=onehot_encode.columns, fill_value=0)
onehot_encode_test = encoder.transform(data_f_masked[['PULocationID', 'DOLocationID']])

In [31]:
np.sqrt(mean_squared_error(data_f_masked['duration_min'].values, model.predict(onehot_encode_test.values)))

AttributeError: 'csr_matrix' object has no attribute 'values'